# Import Useful Modules 

In [1]:
import tensorflow
config = tensorflow.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 #(misalnya kalo pengen 0.4 dari GPU memory)
session = tensorflow.Session(config=config)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import gmtime, strftime
import time
import datetime
from collections import Counter
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# # Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Embedder
from gensim.models import FastText

# Classifier
# from sklearn.svm import SVC
# from sklearn.ensemble import GradientBoostingClassifier as GBC
# from sklearn.linear_model import LogisticRegression as LR
# from sklearn.ensemble import RandomForestClassifier as RFC
# from sklearn.grid_search import GridSearchCV as GS
from xgboost import XGBClassifier as XGB
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.preprocessing import OneHotEncoder as OHE

# import keras
# import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical
from keras import regularizers

from preprocessing_pipeline import preprocessing

Using TensorFlow backend.


In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
import tqdm

In [5]:
#Determine Model's Save Location

#version=
version="version_7"

In [6]:
if(not(os.path.exists("model/{}".format(version)))):
    os.makedirs("model/{}".format(version))

In [7]:
from preprocessing_pipeline import preprocessing

with open("model/{}/word_embedder_500.pickle".format(version), "rb") as file:
    word_embedder = pickle.load(file)

In [8]:
preprocessor=preprocessing(word_embedder.vector_size,word_embedder)
preprocessor

# ---------------------------------------------------------------------------------------------------------------

# Read Data To Pandas Dataframe

In [9]:
def read_product_title_data():
    product_title_only=pd.read_csv('data/query.csv',header=None)
    product_title_only.dropna(inplace=True,axis=0)
    features=product_title_only[1]
    labels=product_title_only[0]
    
    
#     return (product_title_only[:300000],
#             product_title_only[300000:600000],
#             product_title_only[600000:900000],
#             product_title_only[900000:1200000],
#             product_title_only[1200000:1500000],
#             product_title_only[1500000:1800000],
#             product_title_only[1800000:])
    
    return features,labels

In [10]:
def read_product_title_data_small():
    product_title_only=pd.read_csv('data/big.csv',header=None)
    product_title_only.dropna(inplace=True,axis=0)
    features=product_title_only[1]
    labels=product_title_only[0]
    
#     return (product_title_only[:300000],
#             product_title_only[300000:600000],
#             product_title_only[600000:900000],
#             product_title_only[900000:1200000],
#             product_title_only[1200000:1500000],
#             product_title_only[1500000:1800000],
#             product_title_only[1800000:])
    
    return features,labels

In [64]:
def word_count(sentences):
    counts = dict()
    print("1/1")
    for sentence in sentences:
        for word in sentence:
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
    return counts

def class_count(words):
    counts = dict()
    print("1/1")
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

def getFilteredData(product_title,labels,frequency, N_words, word_length):
    print("1/3")
    result=word_count(product_title)
    print("2/3")
    new_product_title=[]
    for sentence in tqdm.tqdm(product_title):
        new_product_title.append([word for word in sentence if result[word]>=frequency and len(word)>=word_length])
    
    print("3/3")
    new_features=[]
    new_labels=[]
    for index,title in tqdm.tqdm(enumerate(new_product_title)):
        if(len(title)>=N_words):
            new_features.append(title)
            new_labels.append(labels[index])
    
    return new_features,new_labels

def getTfIdf(new_product_title):
    print("1/3")
    concatenated_product_title=[]
    for sentence in tqdm.tqdm(new_product_title):
        concatenated_product_title.append(" ".join(sentence))
    print("2/3")
    cv=CountVectorizer()
    result=cv.fit_transform(concatenated_product_title)
    print("3/3")
    tftransformer = TfidfTransformer(smooth_idf=False)
    final_result=tftransformer.fit_transform(result)
    
    return final_result,cv,tftransformer

def getTfIdfCustom(new_product_title,vocab):
    print("1/3")
    concatenated_product_title=[]
    for sentence in tqdm.tqdm(new_product_title):
        concatenated_product_title.append(" ".join(sentence))
    print("2/3")
    cv=CountVectorizer(vocabulary=vocab)
    result=cv.fit_transform(concatenated_product_title)
    print("3/3")
    tftransformer = TfidfTransformer(smooth_idf=False)
    final_result=tftransformer.fit_transform(result)
    
    return final_result,cv,tftransformer

def getFilteredClasses(product_title,labels,top_N):
    print("1/3")
    sorted_by_value = sorted(class_count(labels).items(), key=lambda kv: kv[1])
    valid_class=[value[0] for value in sorted_by_value[-top_N:]]
    print("2/3")
    product_title=list(product_title)
    new_features=[]
    new_labels=[]
    for index,label in tqdm.tqdm(enumerate(labels)):
        if(label in valid_class):
            new_features.append(label)
            new_labels.append(product_title[index])
    
    return new_features,new_labels

In [51]:
sorted_by_value = sorted(class_count(data_for_embedding[1]).items(), key=lambda kv: kv[1])
valid_class=[value[0] for value in sorted_by_value[-100:]]

1/1


In [12]:
data_for_embedding=read_product_title_data()

In [62]:
nfnf,nlnl=getFilteredClasses(data_for_embedding[0],data_for_embedding[1],100)

1/3
1/1


60706it [00:00, 606965.07it/s]

2/3


2440682it [00:04, 527143.42it/s]


In [ ]:
product_title=[preprocessor.remove_parentheses(value) for value in tqdm.tqdm(data_for_embedding["Product Title"])]

In [15]:
def word_count_label(sentences,labels,target):
    counts = dict()
    print("1/1")
    for index,sentence in enumerate(sentences):
        if(labels[index]==target):
            for word in sentence:
                if word in counts:
                    counts[word] += 1
                else:
                    counts[word] = 1
    return counts

In [16]:
new_data=getFilteredData(product_title,list(data_for_embedding[1]),50,2,2)

1/3
1/1


  0%|          | 8321/2440682 [00:00<02:30, 16210.42it/s]

2/3


100%|██████████| 2440682/2440682 [00:07<00:00, 315962.05it/s]
123975it [00:00, 1239340.46it/s]

3/3


2440682it [00:01, 1434169.83it/s]


In [17]:
fff=new_data[0]
lll=new_data[1]

top_3={}
for label in tqdm.tqdm(set(lll)):
    top_3[label]=[value[0] for value in sorted(word_count_label(fff,lll,label).items(), key=lambda x: x[1])[-10:]]
    
allw=[]
for label in list(top_3):
    for i in top_3[label]:
        allw.append(i)
        
enc=LabelEncoder()
enc.fit(allw)

# nfff=[[enc.transform([w])[0] for w in value if w in allw] for value in tqdm.tqdm(fff)]

# nfff=[value for value in tqdm.tqdm(nfff) if len(value)!=0]

  0%|          | 0/232 [00:00<?, ?it/s]

1/1


  0%|          | 1/232 [00:00<01:31,  2.54it/s]

1/1


  1%|          | 2/232 [00:00<01:17,  2.98it/s]

...

1/1
1/1


100%|█████████▉| 231/232 [00:56<00:00,  4.12it/s]

1/1
1/1


100%|██████████| 232/232 [00:56<00:00,  4.13it/s]


LabelEncoder()

In [18]:
aaaaa=getTfIdfCustom(new_data[0],set(allw))

 17%|█▋        | 318296/1872472 [00:00<00:00, 1591189.40it/s]

1/3


100%|██████████| 1872472/1872472 [00:01<00:00, 1805190.70it/s]


2/3
3/3


In [19]:
# concatenated_product_title=[]
# for sentence in tqdm.tqdm(new_data[0]):
#     concatenated_product_title.append(" ".join(sentence))
    
# oenc=CountVectorizer(vocabulary=set(allw))
# nnf=oenc.fit_transform(concatenated_product_title).toarray()

In [20]:
newnew=[]
for index,value in tqdm.tqdm(enumerate(aaaaa[0].toarray())):
    temp=list(value)#list(preprocessor.vectorize_sentence(new_data[0][index]))
    newnew.append(temp) 
    
llee=LabelEncoder()
wenwen=llee.fit_transform(lll)

1872472it [03:15, 9597.25it/s] 


# ---------------------------------------------------------------------------------------------------------------

# Construct Word Embedder (Using fasttext)

In [30]:
X_train,X_test,y_train,y_test=train_test_split(newnew,wenwen,test_size=0.1)

In [ ]:
print("Start Time : {}\n".format(str(datetime.datetime.now())))
s_time=str(datetime.datetime.now())
baseline_xgb=XGB(n_jobs=-1,silent=True,objective='multi:softprob')
baseline_xgb.fit(X_train,y_train)
base_truth_xgb=y_test
pred_xgb=baseline_xgb.predict(X_test)
print("Finish Time : {}\n".format(str(datetime.datetime.now())))
f_time=str(datetime.datetime.now())

Start Time : 2018-07-27 14:01:37.622119



# ---------------------------------------------------------------------------------------------------------------

In [21]:
nn_X_train,nn_X_test,nn_y_train,nn_y_test=train_test_split(newnew,to_categorical(wenwen),test_size=0.1)

In [23]:
model = Sequential()
model.add(Dense(500, input_shape=(len(nn_X_train[0]),), activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(len(nn_y_train[0]), activation='softmax'))


model.compile(optimizer="Adagrad", loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
len(nn_X_train[0])

1123

In [24]:
history = model.fit([nn_X_train], nn_y_train, initial_epoch=0, epochs=25, batch_size=100, validation_data=([nn_X_test],nn_y_test), shuffle=True)

Train on 1685224 samples, validate on 187248 samples
Epoch 1/25
1685224/1685224 [==============================] - 62s 37us/step - loss: 1.8305 - acc: 0.5584 - val_loss: 1.7499 - val_acc: 0.5696
Epoch 2/25
 805900/1685224 [=============>................] - ETA: 29s - loss: 1.7258 - acc: 0.5723

KeyboardInterrupt: 

In [39]:
pred=model.predict(nn_X_test)

KeyboardInterrupt: 

In [55]:
print(classification_report(model.predict_classes([nn_X_test]),[np.argmax(value) for value in nn_y_test]))

             precision    recall  f1-score   support

          0       0.52      0.65      0.58       302
          1       0.04      0.83      0.08         6
          2       0.73      0.54      0.62      7291
          3       0.78      0.68      0.72      8984
          4       0.34      0.35      0.35       420
          5       0.72      0.57      0.63      7886
          6       0.67      0.61      0.64      8456
          7       0.45      0.47      0.46       535
          8       0.21      0.50      0.30       383
          9       0.46      0.54      0.50       397
         10       0.51      0.55      0.53       800
         11       0.43      0.48      0.45       658
         12       0.58      0.60      0.59       742
         13       0.35      0.45      0.40       444
         14       0.70      0.65      0.68      1182
         15       0.08      1.00      0.15         4
         16       0.81      0.86      0.83       527
         17       0.53      0.47      0.50   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
